## Pulse profile analysis
parameters

In [ ]:
#General 
source : str = '' # http://odahub.io/ontology#AstrophysicalObject
output_dir_figures_name : str = 'figures' # Output folder, it will be complemented by a time tag
read_values : bool = True # Whether it should try to read the values present on disk

# Rebinning options and matrix setup
min_sn_matrix : float = 16 # minimum signal-to-noise ratio for the energy-phase and time-phase matrixes
method_calc_rms : str ='counts' # Method to compute the PF 
nbins : int = 32 # Number of phase bins
only_pulsed : bool = True # To use only the pulsed signal to set the minimum S/N
use_counts : bool = True # if sing counts for rebinning (laternative is rate)
flip : bool = True # If true it starts rebinning from the highest energy
    
#Fit PF
e1_flex : float = 9 # minimum energy to search a flex in PF
e2_flex : float = 20 # maximum energy to search a flex in PF
forced_gaussian_centroids : list = [] #Initial centroid value
forced_gaussian_sigmas : list = [] #Initial sigma value
forced_gaussian_amplitudes : list = [] # Initial amplitude value
poly_deg : int = [-1,-1] # Adaptive if negative
noFe : bool = False # it adds a Gaussian iron line in the fit. set to True if there is no need
threshold_p_value : float = 0.05 # p-values threshold to accept polynomial fit degree
n_high_bins_to_exclude : int = 1 # number of high-energy bins to exclude (from the highest)

#plotting and representation
show_initial_energy_guess : bool = False # If we should show the initial guess in global plot as cyn vetical lines
dump_second_harmonic : bool = True # If we should dump the second harmonic in the latex table

#references
reference_urls : list = [] # reference list
reference_main : str = '' # main paper to derive spectral parameters
reference_main_bib : str = '' #bibte reference key for the table
comment : str = '' # our comment on the paper
iron_line : list = [[], [], []] # iron line parameters, position, width, norm or EW
cyclotron_line : list = [[], [] , []] # cyclotron line parameters, position, width, norm
continuum_comment : str = '' # what continuum is used to fit the spectrum 
flux : list = [0,0,0, ''] # flux in 1e-10 units, emin and emax of the range, absorbed or unabsorbed flux
distance : list = [1,10] # min max distance estimates in kpc

#utils
run_xspec : bool = True # if running the Xspec cell
hm_steps : int = 6000 # number of steps of the MCMC chain

### Imports

In [ ]:
## Importing modules developed for this project:
# eg nustarpipeline process to wrap the nustar analysis for our needs, 
# nustarpipeline utils to collect useful functions
# pyxmmsas (originally for XMM) for spectral fitting

%matplotlib widget
import numpy as np
from astropy.table import Table
import matplotlib.pyplot as plt
from scipy.stats import norm
import shutil
import os, sys
from astroquery.simbad import Simbad
from astropy import units as u
from astropy.coordinates import SkyCoord
from importlib import reload
from astropy.io import fits as pf
from nustarpipeline import process, utils
import pyxmmsas as pysas
import hratio
from scipy import interpolate
import json
from datetime import datetime
import logging
from time import gmtime, strftime    

### logging

In [ ]:
file_handler = logging.FileHandler(filename='nustar_utils_%s.log' % (strftime("%Y-%m-%dT%H:%M:%S", gmtime())))
stdout_handler = logging.StreamHandler(sys.stdout)
handlers = [file_handler] #stdout_handler, 

logging.basicConfig(level=logging.INFO, format=' %(levelname)s - %(message)s', handlers=handlers)
logging.getLogger('').addHandler(logging.StreamHandler()) 

In [ ]:
try:
    import xspec
except:
    run_xspec = False
    print('WARNING: Xspec cannot be imported, avoiding its usage')

In [ ]:
keys = ['source', 'min_sn_matrix', 'method_calc_rms',
    'nbins', 'read_values', 'e1_flex', 'e2_flex',
    'forced_gaussian_centroids',
    'forced_gaussian_sigmas',
    'forced_gaussian_amplitudes',
    'noFe', 
    'threshold_p_value',
    'n_high_bins_to_exclude',
    'dump_second_harmonic',
    'reference_urls', 
    'reference_main', 
    'reference_main_bib',
    'iron_line', 
    'cyclotron_line', 
    'continuum_comment',
    'distance', 
    'hm_steps', 
    'poly_deg']
yaml_parameters={}
for kk in keys:
    yaml_parameters.update({kk:locals()[kk]})  

## output setup

In [ ]:
now = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
mylocation=os.getcwd()
print('mylocation = ' + mylocation)
obsid=mylocation.split('/')[-1]
print('OBSID: '+ obsid)

output_dictionary_name = source.replace(' ', '_').replace('+','p').replace('/','') + '_'+obsid+f'_output_results.json'

if os.path.isfile(output_dictionary_name):
    with open(output_dictionary_name, 'r') as fo:
        output_dictionary = json.load(fo)
else:
    output_dictionary = {}

print(output_dictionary)

output_dictionary.update({'OBSID': obsid,
                         'source' : source,
                         'min_sn' : min_sn_matrix})

#human readable extension for the main dat files
dat_file_extension = '_snr_%03d_nbins_%03d_nexcl_%d_' %(min_sn_matrix,nbins,n_high_bins_to_exclude) + method_calc_rms+'.dat'
def json_dumps(thing):
    return json.dumps(
        thing,
        ensure_ascii=False,
        sort_keys=True,
        indent=None,
        separators=(',', ':'),
    )

from hashlib import blake2b
h = blake2b(digest_size=10)
h.update(json_dumps(yaml_parameters).encode('utf-8'))
para_hash = h.hexdigest()
#A hash to encode the output of MCMC chains (note that fitting is performed each time
print(para_hash)

In [ ]:
# now = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
# output_dir_figures_name = output_dir_figures_name.split('/')[0]+'_'+now

if os.path.isdir(output_dir_figures_name):
    print('folder \'%s\' exist' % output_dir_figures_name)
else:
    os.makedirs(output_dir_figures_name)
    
output_dir_figures = mylocation+'/'+output_dir_figures_name

## Reading matrices and rebinning

In [ ]:
reload(utils)
os.chdir(mylocation+'/obs_lc')
#e_min_orig, e_max_orig, pp_orig, dpp_orig, pp_orig_back, dpp_orig_back = utils.read_and_sum_matrixes('E', -1, use_counts=True, subtract_background=False)

e_min_orig, e_max_orig, pp_orig, dpp_orig, pp_orig_back, dpp_orig_back = utils.read_and_sum_matrixes('E', -1, 
                                                                        use_counts=use_counts, 
                                                                        subtract_background=(not use_counts)
                                                                        , nbins=nbins)

os.chdir(mylocation+'/obs_spec')
ff=pf.open('FPMA_sr_rbn.pi')
exposure=ff[1].header['EXPOSURE']
ontime = np.sum(ff[2].data['STOP']-ff[2].data['START'])
date_obs = ff[1].header['DATE-OBS']
date_end = ff[1].header['DATE-END']
ff.close()

os.chdir(mylocation)

e_min, e_max, pp, dpp, pp_back, dpp_back = utils.rebin_matrix(e_min_orig, e_max_orig, pp_orig, dpp_orig, 
                                          min_sn_matrix,
                                              only_pulsed=only_pulsed, use_counts=use_counts, 
                                           background_matrix=(pp_orig_back, dpp_orig_back), flip = flip)

print('ESPOSURE' ,exposure, 'ONTIME', ontime)
output_dictionary.update({'ONTIME': ontime,
                          'EXPOSURE': exposure,
                          'TSTART': date_obs,
                          'TSTOP': date_end,
                        'threshold_p_value': threshold_p_value})

## Computing the pulsed fraction

In [ ]:
reload(utils)
ee_pulsed, dee_pulsed, pulsed_frac, dpulsed_frac = utils.get_pulsed_fraction(e_min, e_max, pp, dpp, 
                        method_calc_rms=method_calc_rms, 
                        output_file=output_dir_figures+'/pf'+ dat_file_extension,
                        read_pf=read_values,
                        pp_back=pp_back, dpp_back=dpp_back, n_harms=1, verbose=False, level=5e-2, 
                        use_poisson=True, statistics='cstat', plot=False)

In [ ]:
if len(ee_pulsed) != len(e_min):
    read_values=False
    ee_pulsed, dee_pulsed, pulsed_frac, dpulsed_frac = utils.get_pulsed_fraction(e_min, e_max, pp, dpp, 
                        method_calc_rms=method_calc_rms, 
                        output_file=output_dir_figures+'/pf'+ dat_file_extension,
                        read_pf=read_values,
                        pp_back=pp_back, dpp_back=dpp_back, n_harms=1, verbose=False, level=5e-2, 
                        use_poisson=True, statistics='cstat', plot=False)

## Removing the last n\_high\_bins\_to\_exclude

In [ ]:
if n_high_bins_to_exclude >0 and n_high_bins_to_exclude < len(ee_pulsed/2):
    ind_selected = range(0,len(ee_pulsed)-n_high_bins_to_exclude)
else:
    ind_selected = range(0,len(ee_pulsed))

## Model the pulse fraction

In [ ]:
reload(utils)
    
stem = output_dir_figures+'/%s_%s' % (source.replace(' ','_'), obsid)
y_lim = [ np.min(pulsed_frac[ind_selected] -dpulsed_frac[ind_selected])-0.1,
          np.max(pulsed_frac[ind_selected] +dpulsed_frac[ind_selected])+0.1]


pulsed_fit_dict = utils.elaborate_pulsed_fraction(ee_pulsed[ind_selected], 
                                        dee_pulsed[ind_selected], 
                                        pulsed_frac[ind_selected], 
                                        dpulsed_frac[ind_selected],
                                        debug_plots=False,
                                        stem=stem, 
                                        poly_deg = poly_deg, title=source + ' '+ obsid, save_plot=True,
                                        e1=e1_flex, e2=e2_flex, 
                                        forced_gaussian_centroids=forced_gaussian_centroids, 
                                        forced_gaussian_sigmas=forced_gaussian_sigmas,
                                        forced_gaussian_amplitudes=forced_gaussian_amplitudes, noFe = noFe,
                                        y_lim=y_lim, threshold_p_value = threshold_p_value)

In [ ]:
reload(utils)
mcmc_low, corner_low = utils.explore_fit_mcmc(pulsed_fit_dict['pulsed_fit_low'], pars=[], 
                                              hm_steps=hm_steps, 
                                              hm_walkers=50, hm_jump=int(hm_steps/10), 
                                              plot_corner=True, print_file=True, high_part=False,
                                              stem=stem, read=read_values, para_hash=para_hash)


In [ ]:
reload(utils)
if pulsed_fit_dict['pulsed_fit_high'] is not None:
    
    mcmc_high, corner_high = utils.explore_fit_mcmc(pulsed_fit_dict['pulsed_fit_high'], pars=[], 
                                                    hm_steps=hm_steps, 
                                                    hm_walkers=50, hm_jump=int(hm_steps/10), 
                                                    plot_corner=True, print_file=True, high_part=True,
                                                    stem=stem, read=read_values, para_hash=para_hash)
    
    output_dictionary.update({'mcmc_high_results':stem+'.json',
                              'mcmc_high_chain':stem+'.fits'})

## Compute the phases and amplitudes of the first harmonics

In [ ]:
reload(utils)
As,dAs,phases,dphases,A2s,dA2s,phases2,dphases2= utils.get_phases(pp, dpp, 
               ind_selected, ee_pulsed, dee_pulsed, plot=True,
               margin=0.5, debug=True, distance_factor=2, 
               output_figure_file=None, 
               output_file=output_dir_figures+'/phases' + dat_file_extension, read=read_values)

## Modelling the first harmonic

In [ ]:
reload(utils)
stem2 = output_dir_figures+'/%s_%s_1rst_armonic' % (source.replace(' ','_'), obsid)
As = np.array(As)
dAs = np.array(dAs)
y_lim2 = [min(As)-min(As)-0.1, max(As)+max(As)+0.1]

pulsed_fit_first_dict = utils.elaborate_pulsed_fraction(ee_pulsed[ind_selected], 
                                        dee_pulsed[ind_selected], As, dAs,
                                        debug_plots=False,
                                        stem=stem2, 
                                        poly_deg = poly_deg, title=source + ' '+ obsid, save_plot=True,
                                        e1=e1_flex, e2=e2_flex, 
                                        forced_gaussian_centroids=forced_gaussian_centroids, 
                                        forced_gaussian_sigmas=forced_gaussian_sigmas,
                                        forced_gaussian_amplitudes=forced_gaussian_amplitudes, noFe = noFe,
                                        y_lim=y_lim2, threshold_p_value = threshold_p_value, 
                                                        ylabel='$A_1$' )

In [ ]:
reload(utils)
mcmc_low_first, corner_low_first = utils.explore_fit_mcmc(pulsed_fit_first_dict['pulsed_fit_low'], 
                                                          pars=[], hm_steps=hm_steps, 
                                              hm_walkers=50, hm_jump=int(hm_steps/10), 
                                              plot_corner=True, print_file=True, high_part=False,
                                              stem=stem2, read=read_values, para_hash=para_hash)

In [ ]:
if pulsed_fit_first_dict['pulsed_fit_high'] is not None:
    mcmc_high_first, corner_high_first = utils.explore_fit_mcmc(pulsed_fit_first_dict['pulsed_fit_high'], 
                                                            pars=[], hm_steps=hm_steps, 
                                              hm_walkers=50, hm_jump=int(hm_steps/10), 
                                              plot_corner=True, print_file=True, high_part=True,
                                              stem=stem2, read=read_values, para_hash=para_hash)

## Modelling the second harmonics

In [ ]:
stem3 = output_dir_figures+'/%s_%s_2nd_armonic' % (source.replace(' ','_'), obsid)
A2s = np.array(A2s)
dA2s = np.array(dA2s)

y_lim3 = [min(A2s)-min(A2s)*0.1,max(A2s)+max(A2s)*0.1]

pulsed_fit_second_dict = utils.elaborate_pulsed_fraction(ee_pulsed[ind_selected], 
                                                        dee_pulsed[ind_selected], 
                                                                  A2s, 
                                                                  dA2s,
                                                    debug_plots=False,
                                        stem=stem3, 
                                        poly_deg = poly_deg, title=source + ' '+ obsid, save_plot=True,
                                        e1=e1_flex, e2=e2_flex, 
                                        forced_gaussian_centroids=forced_gaussian_centroids, 
                                        forced_gaussian_sigmas=forced_gaussian_sigmas,
                                        forced_gaussian_amplitudes=forced_gaussian_amplitudes, noFe = noFe,
                                        y_lim=y_lim3, threshold_p_value = threshold_p_value, 
                                                         ylabel='$A_2$')

In [ ]:
mcmc_low_second, corner_low_second = utils.explore_fit_mcmc(pulsed_fit_second_dict['pulsed_fit_low'], pars=[], 
                                                            hm_steps=hm_steps, 
                                              hm_walkers=50, hm_jump=int(hm_steps/10), 
                                              plot_corner=True, print_file=True, high_part=False,
                                              stem=stem3, read=read_values, para_hash=para_hash)

In [ ]:
if pulsed_fit_second_dict['pulsed_fit_high'] is not None:
    mcmc_high_second, corner_high_second = utils.explore_fit_mcmc(pulsed_fit_second_dict['pulsed_fit_high'], 
                                                              pars=[], hm_steps=hm_steps, 
                                              hm_walkers=50, hm_jump=int(hm_steps/10), 
                                              plot_corner=True, print_file=True,high_part=True,
                                              stem=stem3, read=read_values, para_hash=para_hash)

## Preparing a dictionary for output

In [ ]:
###### reload(pysas)
reload(utils)

from nustarpipeline import data_manipulation
reload(data_manipulation)

dict_param_harms = {}


x1s= [mcmc_low, mcmc_low_first]
if pulsed_fit_dict['pulsed_fit_high'] is not None:
    x2s= [mcmc_high, mcmc_high_first]
x3s= [pulsed_fit_dict, pulsed_fit_first_dict]
    
names = ['PF', '$1^\\mathrm{st}$']

if dump_second_harmonic:
    x1s.append(mcmc_low_second)
    if pulsed_fit_dict['pulsed_fit_high'] is not None:
        x2s.append(mcmc_high_second)
    x3s.append(pulsed_fit_second_dict)
    names.append('$2^\\mathrm{nd}$')
               

if pulsed_fit_dict['pulsed_fit_high'] is not None:
    for x1,x2, x3 ,nn in zip(x1s, x2s, x3s, names):
        t1 = data_manipulation.lmfit_chain_to_dict('Low '+ nn, x1, high_part=False, cstat_obj=x3['pulsed_fit_low'])

        t2 = data_manipulation.lmfit_chain_to_dict('High '+nn, x2, high_part=True, cstat_obj=x3['pulsed_fit_high'])
        dict_param_harms.update(data_manipulation.merge_dicts(nn, [t1,t2], patterns_to_exclude = ['poly_']))


        dict_param_harms[nn].update({'e_turn': [x3['e_turn']]*3,
                                    'deg_low' : [x3['deg_poly_low']]*3,
                                    'deg_high' : [x3['deg_poly_high']]*3})
        
else:
    
    for x1, x3 ,nn in zip(x1s, x3s, names):
        t1 = data_manipulation.lmfit_chain_to_dict('Low '+ nn, x1, high_part=False, cstat_obj=x3['pulsed_fit_low'])

        dict_param_harms.update(data_manipulation.merge_dicts(nn, [t1], patterns_to_exclude = ['poly_']))

        dict_param_harms[nn].update({'e_turn': [x3['e_turn']]*3,
                                    'deg_low' : [x3['deg_poly_low']]*3})
    
    
if noFe:
    dict_param_harms.update(data_manipulation.literature_to_dict('Spectral', None, cyclotron_line))
else:
    dict_param_harms.update(data_manipulation.literature_to_dict('Spectral', iron_line, cyclotron_line))


## Dumping the latex table of results

In [ ]:
out_str = pysas.dump_latex_table(dict_param_harms, data_manipulation.mcmc_latex_dict, 
                                 vertical_stack=False, to_skip=['poly_'])

output_dictionary.update(dict_param_harms)

#print(out_str)
out_fname = output_dir_figures + '/%s_%s_harms_table.tex' % (source.replace(' ','_'), obsid)
with open(out_fname,'w') as ff:
    ff.write('\\begin{table*}\n')
    ff.write('\\label{tab:%s_line_parameters}\n'% source.replace(' ','_').lower())
    ff.write('\caption{Best-fit parameters of the %s ObsID %s pulse and amplitude models compared to spectral results. \citep{%s}.}\n' % (source,obsid,reference_main_bib))
    ff.write('\\centering\n')
    ff.write(out_str)
    ff.write('''\\tablefoot{
    $\\chi^2_{\mathrm{red,lo}}$ and $\\chi^2_{\\mathrm{red},hi}$ are the reduced $\\chi^2$ fo the lower-
    and higher-energy sections, respectively, with
    $n_\\mathrm{pol}^\\mathrm{(lo)}$ and $n_\mathrm{pol}^\\mathrm{(hi)}$ the corresponding polynomial orders. 
    $E_\\mathrm{split}$ is the energy at which we separate the two regions.\n\t}
    ''')
    ff.write('\\end{table*}\n')
with open(out_fname,'r') as ff:
    for ss in ff.readlines():
        print(ss[0:-1])

## Dumping the table in HTML format

In [ ]:
reload(pysas)
reload(utils)

#First, we need to change the names of dictionary items to display them in html
translate_dict = {'PF': 'PF', 
                  '$1^\\mathrm{st}$' : '1<sup>st</sup>', 
                  '$2^\\mathrm{nd}$' : '2<sup>nd</sup>', 
                  'Spectral': 'Spectral'}

dict_param_harms_html = {}

for kk, ii in dict_param_harms.items():
    dict_param_harms_html.update( {translate_dict[kk] : ii} )

    
out_str = pysas.dump_html_table(dict_param_harms_html, utils.mcmc_html_dict, 
                                 vertical_stack=False, to_skip=['poly_'], table_class='ppanda')


#print(out_str)
out_fname = output_dir_figures + '/%s_%s_harms_table.html' % (source.replace(' ','_'), obsid)
with open(out_fname,'w') as ff:
    ff.write(out_str)
with open(out_fname,'r') as ff:
    for ss in ff.readlines():
        print(ss[0:-1])



## Compute lags and correlation

In [ ]:
reload(utils)
lag, lag_error, correlation, correlation_error = utils.get_cross_correlation_with_error(
                                                                pp[ind_selected,:], dpp[ind_selected,:], 
                                                                ee_pulsed=ee_pulsed[ind_selected],
                                                                dee_pulsed=dee_pulsed[ind_selected], 
                                                                margin=0.2, 
                                                                n_to_fit=3, title=source+ ' ' + obsid,
                                                                read=read_values, 
                                                                output_file=output_dir_figures+'/lags' + dat_file_extension,
                                                                stem=output_dir_figures+'/%s_%s' % (source.replace(' ','_'), obsid))

### plot problematic points in debug mode
to see if thre aissues wih the lags

In [ ]:
if False:
    debug_indexes = np.where(lag_error > 0.05)[0]
    if len(debug_indexes>0):
        lag, lag_error, correlation, correlation_error = utils.get_cross_correlation_with_error(
                                                                pp[ind_selected,:], dpp[ind_selected,:], 
                                                                ee_pulsed=ee_pulsed[ind_selected],
                                                                dee_pulsed=dee_pulsed[ind_selected], margin=0.2, 
                                                                n_to_fit=3, title=source+ ' ' + obsid,
                                                                stem=output_dir_figures+'/%s_%s' % (source.replace(' ','_'), obsid),
                                                                output_file=output_dir_figures+'/lags' + dat_file_extension,
                                                                debug_indexes=debug_indexes)


## Make the global plot

In [ ]:
reload(utils)

i=1
e_cyc = []

if pulsed_fit_dict['pulsed_fit_high'] is not None:
    while 'g%d_center' % i in pulsed_fit_dict['pulsed_fit_high'].best_values:
        e_cyc.append(pulsed_fit_dict['pulsed_fit_high'].best_values['g%d_center'%i])
        e_cyc.append(pulsed_fit_dict['pulsed_fit_high'].best_values['g%d_sigma'%i])
        i+=1
else:
    if noFe is False:
        i=2
    while 'g%d_center' % i in pulsed_fit_dict['pulsed_fit_low'].best_values:
        e_cyc.append(pulsed_fit_dict['pulsed_fit_low'].best_values['g%d_center'%i])
        e_cyc.append(pulsed_fit_dict['pulsed_fit_low'].best_values['g%d_sigma'%i])
        i+=1

if noFe is False:
    e_iron = [pulsed_fit_dict['pulsed_fit_low'].best_values['g1_center'],   
              pulsed_fit_dict['pulsed_fit_low'].best_values['g1_sigma'] ]
else:
    e_iron=None

if show_initial_energy_guess:
    initial_values = forced_gaussian_centroids
else:
    initial_values = None

_=utils.plot_all_paper(pp, dpp, ee_pulsed,dee_pulsed,
                       pulsed_frac,dpulsed_frac,
                   pulsed_fit_dict['e_turn'], 
                       pulsed_fit_dict['pulsed_fit_low'], 
                       pulsed_fit_dict['pulsed_fit_high'], noFe, initial_values,
                   correlation, correlation_error,lag,lag_error,
                    np.array([As,dAs]) ,
                     np.array([phases, dphases]) ,
                     np.array([A2s, dA2s]),
                     np.array([phases2,dphases2]),
                     e_iron,e_cyc,
                       stem = output_dir_figures+'/'+source.replace(' ','_')+'_' + obsid,
                       title=source + ' ' + obsid,
                  ind_selected=ind_selected, scale='log', source=source + ' in ObsID ' + obsid )

## Make a basic spectral model

In [ ]:
if run_xspec:
    import xspec
    from IPython.display import Image
    from IPython.display import display
    os.chdir(mylocation+'/obs_spec')

    if not os.path.isfile('files-auto.xcm'):
        status = utils.make_basic_fit()
    else:
        status = utils.run('xspec - files-auto.xcm')
    source_trimmed = source.replace(' ', '_').replace('+','p').replace('/','')
    pysas.epic_xspec_mcmc_fit(xspec, 'mod_base.xcm', 
                                pn_spec="FPMA_sr_rbn.pi",
                                mos1_spec="FPMB_sr_rbn.pi",
                                mos2_spec="none",
                                ignore_string=2* ['**-3.0,70.0-**'],
                                outputfiles_basename=source_trimmed+"-gw-base-", 
                                run_chain=False,compute_errors=False)
    xspec.AllModels.calcFlux("3 70.0")
    rate = xspec.AllData(1).rate
    flux = xspec.AllData(1).flux
    ff = pf.open("FPMA_sr_rbn.pi")
    output_dictionary.update({'rate' : rate,
                              'flux': flux,
                              'exposure': ff[1].header['EXPOSURE'],
                              'tstart' : ff[1].header['DATE-OBS'].replace('T', ' '),
                               'tstop' : ff[1].header['DATE-END'].replace('T', ' ')
                               })
    ff.close()
    # if status == 0:    
    #     _ = display(Image(filename='basic-plot.gif_2', format="gif"))
    #     shutil.move('basic-plot.gif_2', 'spec_%s_%s.gif' % (source_trimmed, obsid))
    #     utils.run('convert ' + 'spec_%s_%s.gif ' % (source_trimmed, obsid) +
    #               'spec_%s_%s.png' % (source_trimmed, obsid))
    os.chdir(mylocation)

In [ ]:
reload(utils)
period_formatted = utils.get_period_formatted()
raw_period = utils.get_period_formatted(raw=True)
output_dictionary.update({'period_formatted': period_formatted,
                          'period': raw_period[0],
                          'period uncertainty': raw_period[1]})

## Dump the result dictionary

In [ ]:
import yaml
with open(output_dictionary_name, 'w') as fo:
    json.dump(output_dictionary, fo)
with open('model_para.yaml', 'w') as yf:
        yaml.dump(yaml_parameters, yf)

In [ ]:
result_file = output_dictionary_name # oda:results